# Simulation Normalized Node Batch


## input arguments

In [1]:
startmonth = 1

#define how the nodes will grow
node_growth="0:5,6:150,12:1000,18:2000,24:8000,36:12000,48:20000,60:20000"
# node_growth="0:5,6:150,12:1000,24:5000"
# node_growth="0:1000"

#tft price in 5 years, has impact on return
tft_price=3

#bill of material name
# bom_name = "CH_archive"
bom_name = "A_dc_rack"
# bom_name = "hpe_dl385_amd"
# bom_name = "amd"

from lib import *

simulation = j.tools.tfgrid_simulator.simulation_get(
            name="default",
            tokencreator_name="optimized",
            bom_name="amd", #dont change here, this is the growth calc
            node_growth=node_growth,
            tft_growth=tft_price,
            reload=True,
        )

#parameters for simulation
# choose your hardware profile (other choices in stead of amd or supermicro or hpe)
bom, environment2 = j.tools.tfgrid_simulator.bom_environment_get(bom_name)
nb = simulation.nodesbatch_get_environment(month=startmonth, environment=environment2)
server=environment2.node_normalized


- To change token creation params [default.py](/edit/params/default.py)
- To edit token creation open [token_creators](/tree/token_creators)

## Node Overview (= 1 server)

In [2]:
server


## threefold.simulation.device
 - name                : hpe_storage_server
 - description         : 
 - device_template_name: hpe_storage_server
 - cost                : 55,574
 - power               : 1,004
 - rackspace_u         : 4
 - cru                 : 32
 - sru                 : 5760
 - hru                 : 672000
 - mru                 : 256
 - su                  : 608
 - cu                  : 63
 - cu_passmark         : 377
 - su_perc             : 90.98%
 - cu_perc             : 9.02%
 - cpr                 : 703
 - cost_su_capex       : 83.2
 - cost_cu_capex       : 78.7
 - cost_power          : 108
 - cost_rack           : 48
 - cost_month          : 1,082
 - cost_cu_month       : 1.5322397146735522
 - cost_su_month       : 1.6200412031626554


In [3]:
environment2.devices.compute

(11,
 
 ## threefold.simulation.device
  - name                : hpe_compute_server
  - description         : 
  - device_template_name: hpe_compute_server
  - cost                : 20,120
  - power               : 698
  - rackspace_u         : 2
  - cru                 : 64
  - sru                 : 7680
  - hru                 : 96000
  - mru                 : 512
  - su                  : 144
  - cu                  : 127
  - cu_passmark         : 283
  - su_perc             : 57.46%
  - cu_perc             : 42.54%
  - cpr                 : 335
  - cost_su_capex       : 80.3
  - cost_cu_capex       : 67.0
  - cost_power          : 75.4
  - cost_rack           : 24
  - cost_month          : 434
  - cost_cu_month       : 1.4477442092390211
  - cost_su_month       : 1.7345000736322804
 ,
 'n')

In [4]:
if hasattr(environment2.devices,"storage"):
    environment2.devices.storage

In [5]:
environment2


## threefold.simulation.environment
 - name                : A_dc_rack
 - description         : 
 - cost_power_kwh      : 0.15
 - cost_rack_unit      : 12
 - cost                : 508,190
 - power               : 12,898
 - rackspace_u         : 44
 - cru                 : 864
 - sru                 : 113280
 - hru                 : 4416000
 - mru                 : 6912
 - su                  : 4624
 - cu                  : 1724
 - cu_passmark         : 4,998
 - su_perc             : 76.12%
 - cu_perc             : 23.88%
 - cpr                 : 7,210
 - cost_su_capex       : 83.7
 - cost_cu_capex       : 70.4
 - cost_power          : 1,392
 - cost_rack           : 528
 - cost_month          : 10,390
 - cost_cu_month       : 1.4394545381615664
 - cost_su_month       : 1.7102898737477206
 - nr_devices          : 18
 - nr_nodes            : 16
 - sales_price_total   : 0
 - sales_price_cpr_unit: 0


In [6]:
md(f"""
### Device Details

- nr of TB netto usable = {round(server.su,0)} TB
- nr of cpu cores = {round(server.cru,1)} Cores
- power used = {server.power} watt
- performance in passmark = {round(server.cu_passmark*server.cu)}

""")


### Device Details

- nr of TB netto usable = 608.0 TB
- nr of cpu cores = 32.0 Cores
- power used = 1004 watt
- performance in passmark = 24034



In [7]:
nb.simulated_months=[]
nb._data

## threefold.simulation.nodesbatch
 - name                : custom
 - batch_nr            : 1
 - nrnodes             : 16
 - month_start         : 1
 - months_left         : 60
 - tft_farmed_before_simulation: 0

    ## threefold.simulation.nodesbatch.node
     - rackspace_u         : 4
     - cost_hardware       : 55,574
     - cpr                 : 403
     - power               : 1004
    

 - simulated_months    : []


In [8]:
md(f"""
- Hardware Investment Cost for the node batch : USD {nb.cost_hardware}
- ROI In Months                 : {nb.roi_months}
""")


- Hardware Investment Cost for the node batch : USD 889,184
- ROI In Months                 : 14


In [9]:
nb.graph_tft(single=True)

FigureWidget({
    'data': [{'name': 'farmed',
              'type': 'scatter',
              'uid': '86e736cf…

In [10]:
nb.graph_usd(single=True)

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'farmed',
              'type': 'scat…

In [11]:
nb.graph_usd(single=True,cumul=True)

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'farmed',
              'type': 'scat…

In [1]:

md(f"""
- The Return On investment is compared to cost to buy all the hardware.
- The costs of the rackspace and power have been paid with income of TFT, so its the net result which is shown.
- ROI is in **{nb.roi_months}** months (starting month:{startmonth}).
- the ROI after 60 months' is:  **{round(nb.roi_end,1)}**
""")

NameError: name 'md' is not defined

### Token price per month

In [13]:
simulation.graph_token_price()

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '22e104db-c611-4afd-854d-cf98ad425f99',
 …

In [14]:
simulation.graph_nr_nodes()

FigureWidget({
    'data': [{'name': 'new nodes',
              'type': 'scatter',
              'uid': 'c3916…